### Initialize ellipsoids with DPD

In [1]:
import gsd, gsd.hoomd 
import hoomd
import mbuild as mb
import numpy as np
import scipy.stats
import unyt as u
import warnings
import freud
from flowermd.base import Pack,Lattice, Simulation,BaseHOOMDForcefield,Polymer
from flowermd.base.system import System
from flowermd.utils.constraints import create_rigid_ellipsoid_chain
from scipy.spatial.distance import pdist
warnings.filterwarnings('ignore')

/Users/stephaniemccallum/miniforge3/envs/mupt-examples/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [25]:
class EllipsoidChainRand(Polymer):
    def __init__(
        self,
        lengths,
        num_mols,
        lpar,
        bead_mass,
        density,
        bond_L=0.1, #T-T bond length
        name="ellipsoid_chain",
    ):
        self.bead_mass = bead_mass
        self.lpar = lpar
        self.bond_L = bond_L
        self.density = density
        N=lengths*num_mols
        L = np.cbrt(N/ self.density)
        self.L = L
        self.bead_constituents_types = ["X", "A", "T", "T"]
        super(EllipsoidChainRand, self).__init__(
            lengths=lengths, num_mols=num_mols, name=name
        )

    def _build(self, length):
        bead = mb.Compound(name="ellipsoid")
        center = mb.Compound(pos=(0, 0, 0), name="X", mass=self.bead_mass / 4)
        head = mb.Compound(
            pos=(0, 0, self.lpar + (self.bond_L / 2)),
            name="A",
            mass=self.bead_mass / 4,
        )
        tether_head = mb.Compound(
            pos=(0, 0, self.lpar), name="T", mass=self.bead_mass / 4
        )
        tether_tail = mb.Compound(
            pos=(0, 0, -self.lpar), name="T", mass=self.bead_mass / 4
        )
        bead.add([center, head, tether_head, tether_tail])
        bead.add_bond([center, head])

        chain = mb.Compound()
        last_bead = None
        rand_range = ((self.L/2) - (self.lpar+(self.bond_L / 2))) #reducing step size for random walk
        print('range',rand_range)
        for i in range(length):
            translate_by = np.random.uniform(low=-1, high=1, size=(3,))
            translate_by /= np.linalg.norm(translate_by)*self.bond_L
            this_bead = mb.clone(bead)
            if last_bead:
                chain.add_bond([this_bead.children[0], last_bead.children[1]])
                chain.add_bond([this_bead.children[3], last_bead.children[2]])
                this_bead.translate(by=self.pbc(translate_by+last_bead.pos,pos_range=([rand_range]*3)))
            else:
                translate_by = np.random.uniform(low=-rand_range, high=rand_range, size=(3,))
                this_bead.translate(by=self.pbc(translate_by,pos_range=([rand_range]*3)))
            chain.add(this_bead)
            last_bead = this_bead
        chain.name = f"{self.name}_{length}mer"
        return chain

    def pbc(self,d,pos_range):
        for i in range(3):
            while d[i] > pos_range[i] or d[i] < -(pos_range[i]):
                if d[i] < -pos_range[i]:
                    d[i] += pos_range[i]
                if d[i] > pos_range[i]:
                    d[i] -= pos_range[i]
        return d

class DPD_FF(BaseHOOMDForcefield):
    def __init__(
        self,
        epsilon,
        lpar,
        lperp,
        A,
        gamma,
        kT,
        r_cut,
        angle_k=None,
        angle_theta0=None,
        bond_k=100,
        bond_r0=1.1,
    ):
        self.epsilon = epsilon
        self.lpar=lpar
        self.lperp=lperp
        self.gamma = gamma
        self.A = A
        self.kT = kT
        self.r_cut = r_cut
        self.angle_k = angle_k
        self.angle_theta0 = angle_theta0
        self.bond_k = bond_k
        self.bond_r0 = bond_r0
        hoomd_forces = self._create_forcefield()
        super(DPD_FF, self).__init__(hoomd_forces)

    def _create_forcefield(self):
        forces = []
        # Bonds
        bond = hoomd.md.bond.Harmonic()
        bond.params["T-T"] = dict(k=self.bond_k, r0=self.bond_r0)
        bond.params["A-X"] = dict(k=0, r0=0)
        forces.append(bond)
        # Angles
        if all([self.angle_k, self.angle_theta0]):
            angle = hoomd.md.angle.Harmonic()
            angle.params["X-A-X"] = dict(k=self.angle_k, t0=self.angle_theta0)
            angle.params["A-X-A"] = dict(k=0, t0=0)
            forces.append(angle)
        # DPD Pairs
        nlist = hoomd.md.nlist.Cell(buffer=0.40, exclusions=["body"])
        dpd = hoomd.md.pair.DPD(nlist=nlist,kT=self.kT,default_r_cut=self.r_cut)
        dpd.params[("X", "X")] = dict(A=self.A, gamma=self.gamma)
        # Add zero pairs
        for pair in [
            ("R", "R"),
            ("T", "T"),
            ("T", "R"),
            ("A", "A"),
            ("A", "X"),
            ("A", "T"),
            ("A", "R"),
            ("X", "R"),
            ("X", "T"),
        ]:
            dpd.params[pair] = dict(A=0,gamma=0.1)
            dpd.params[pair].r_cut = 0.0
        forces.append(dpd)
        return forces

class RandomSystem(System):
    def __init__(self, molecules, base_units=dict()):
        self.L = molecules.L
        super(RandomSystem, self).__init__(
            molecules=molecules, base_units=base_units
        )
    def _build_system(self):
        chain = self.all_molecules
        comp = mb.Compound()
        comp.add(chain)
        comp.box = mb.Box(lengths=np.array([self.L] * 3))
        return comp
        
def pbc(d,box):
    '''
    periodic boundary conditions
    
    '''
    for i in range(3):
        a = d[:,i]
        pos_max = np.max(a)
        pos_min = np.min(a)
        while pos_max > box[i]/2 or pos_min < -box[i]/2:
            a[a < -box[i]/2] += box[i]
            a[a >  box[i]/2] -= box[i]
            pos_max = np.max(a)
            pos_min = np.min(a)
    return d
    
def check_bond_length_equilibration(pos,num_mon,num_pol,box,max_bond_length=1.1,min_bond_length=0.95):
    '''
    Check the bond distances of T-T particles.
    
    '''
    frame_ds = []
    for j in range(num_pol):
        idx = j*num_mon
        d1 = pos[idx:idx+num_mon-1] - pos[idx+1:idx+num_mon]
        bond_l = np.linalg.norm(pbc(d1,box),axis=1)
        frame_ds.append(bond_l)
    max_frame_bond_l = np.max(np.array(frame_ds))
    min_frame_bond_l = np.min(np.array(frame_ds))
    print("max: ",max_frame_bond_l," min: ",min_frame_bond_l)
    if max_frame_bond_l <= max_bond_length and min_frame_bond_l >= min_bond_length:
        print("Bonds relaxed.")
        return True
    if max_frame_bond_l > max_bond_length or min_frame_bond_l < min_bond_length:
        return False

def check_inter_particle_distance(positions,box,minimum_distance=0.95):
    '''
    Check particle separations.
    
    '''
    aq = freud.locality.AABBQuery(box,positions)
    aq_query = aq.query(
        query_points=positions,
        query_args=dict(r_min=0.0, r_max=minimum_distance, exclude_ii=True),
    )
    nlist = aq_query.toNeighborList()
    if len(nlist)==0:
        print("Inter-particle separation reached.")
        return True
    else:
        return False

In [40]:
lengths=10
num_mols=10
chains = EllipsoidChainRand(lengths=lengths,num_mols=num_mols,lpar=0.5,bead_mass=1.0,density=0.85)
ff = DPD_FF(epsilon=1.0,lpar=0.5,lperp=0.5,A=5000,gamma=800,kT=2.0,r_cut=1.15,bond_k=30000,bond_r0=0.1)
ff.hoomd_forces
system = RandomSystem(molecules=chains)
system.to_gsd('test.gsd')
gsd_path=('ellipsoid-test.gsd')
rigid_frame, rigid = create_rigid_ellipsoid_chain(
    system.hoomd_snapshot,lpar=0.5,lperp=0.5
)
sim = Simulation(
    initial_state=rigid_frame,
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    dt=0.0003,
    gsd_write_freq=int(1000),
    gsd_file_name='trajectory.gsd',
    log_write_freq=int(1000),
    log_file_name='log.txt')

sim.save_restart_gsd()
sim.run_NVT(n_steps=0, kT=2.0, tau_kt=10*sim.dt)
sim.flush_writers()

box = sim.box_lengths

with gsd.hoomd.open('trajectory.gsd', "r") as traj:
    snap = traj[-1]
    typids = snap.particles.typeid
    r_idx = np.where(typids == 2)
    positions = snap.particles.position[r_idx[0]]
    
while not check_bond_length_equilibration(positions,num_mon=lengths*2,num_pol=num_mols,box=box,max_bond_length=2.15,min_bond_length=0.9):
    sim.run_NVT(n_steps=10000, kT=2.0, tau_kt=10*sim.dt)
    sim.flush_writers()
    with gsd.hoomd.open('trajectory.gsd', "r") as traj:
        snap = traj[-1]
        r_idx = np.where(typids == 2)
        positions = snap.particles.position[r_idx[0]]
        
with gsd.hoomd.open('trajectory.gsd', "r") as traj:
    snap = traj[-1]
    typids = snap.particles.typeid
    r_idx = np.where(typids == 0)
    positions = snap.particles.position[r_idx[0]]
    
while not check_inter_particle_distance(positions,box,minimum_distance=0.9):
    sim.run_NVT(n_steps=10000, kT=2.0, tau_kt=10*sim.dt)
    sim.flush_writers()
    with gsd.hoomd.open('trajectory.gsd', "r") as traj:
        snap = traj[-1]
        r_idx = np.where(typids == 0)
        positions = snap.particles.position[r_idx[0]]

range 1.899986525148223
range 1.899986525148223
range 1.899986525148223
range 1.899986525148223
range 1.899986525148223
range 1.899986525148223
range 1.899986525148223
range 1.899986525148223
range 1.899986525148223
range 1.899986525148223
Initializing simulation state from a gsd.hoomd.Frame.
max:  3.7775702  min:  0.62294257
Step 1000 of 10000; TPS: 1940.51; ETA: 0.1 minutes
Step 2000 of 10000; TPS: 2312.65; ETA: 0.1 minutes
Step 3000 of 10000; TPS: 2616.08; ETA: 0.0 minutes
Step 4000 of 10000; TPS: 2875.92; ETA: 0.0 minutes
Step 5000 of 10000; TPS: 3065.0; ETA: 0.0 minutes
Step 6000 of 10000; TPS: 3206.85; ETA: 0.0 minutes
Step 7000 of 10000; TPS: 3320.42; ETA: 0.0 minutes
Step 8000 of 10000; TPS: 3413.13; ETA: 0.0 minutes
Step 9000 of 10000; TPS: 3494.39; ETA: 0.0 minutes
max:  2.1251886  min:  0.99999964
Bonds relaxed.
Step 0 of 10000; TPS: 0.0; ETA: nan hours, nan minutes
Step 1000 of 10000; TPS: 4259.03; ETA: 0.0 minutes
Step 2000 of 10000; TPS: 4230.67; ETA: 0.0 minutes
Step 300

# Getting orientation vectors and 'T' anchor positions from trajectory

In [41]:
snap = sim.state

In [42]:
traj = gsd.hoomd.open('trajectory.gsd','r')

In [43]:
last_frame = traj[-1]

In [44]:
orientations = last_frame.particles.orientation

In [45]:
N = 16
print(last_frame.particles.typeid, last_frame.particles.types)
print("orientations of rigid bodies\n",orientations[0:N])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 1 2 2 3 1 2 2 3 1 2
 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2
 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3
 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1
 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2
 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2
 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3
 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1
 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2
 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2
 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3
 1 2 2 3 1 2 2 3 1 2 2 3 

In [46]:
#positions of T particles
positions = last_frame.particles.position
typids = last_frame.particles.typeid
print(last_frame.particles.types)
t_idx = np.where(typids == 2)

['R', 'A', 'T', 'X']


In [47]:
t_positions = positions[t_idx[0]]

In [48]:
print('ellipsoid anchor positions',t_positions)

ellipsoid anchor positions [[ 0.4330578  -0.8685485   2.3048892 ]
 [-0.45998392 -1.2668575   2.0955417 ]
 [ 0.9429449  -0.15516576 -1.9199108 ]
 [ 0.49429902 -0.79689556  2.3580494 ]
 [ 0.98945236  0.31648824 -0.9046432 ]
 [ 0.9655338  -0.08989806 -1.8180314 ]
 [ 1.3163595   0.41563866  0.14448822]
 [ 0.9681169   0.32191092 -0.78821874]
 [ 1.4107416   0.7761575   1.1547228 ]
 [ 1.3769958   0.48702005  0.19803017]
 [ 0.7791712   1.4106754   1.7932497 ]
 [ 1.332785    0.8019942   1.2249023 ]
 [ 0.0374314   1.7346044  -2.3842318 ]
 [ 0.75422955  1.482144    1.8657687 ]
 [-1.0523087   1.6873332   2.414086  ]
 [-0.05824398  1.6875716  -2.3770964 ]
 [-1.5716884   1.9056177   1.4750857 ]
 [-1.16068     1.7116262   2.3658378 ]
 [-1.3329723   2.2752023   0.45183945]
 [-1.542874    1.94999     1.3738906 ]
 [ 1.0123838   1.5512877   0.45015386]
 [ 1.922538    1.9452295   0.57832986]
 [ 0.06123244  1.0246414   0.6362493 ]
 [ 0.90757     1.5136387   0.4250758 ]
 [-0.36465594  0.532736    1.5051519 

# Visualizing Ellipsoids
You must add particle shape information to the gsd file for Ovito to read

In [49]:
from cmeutils.gsd_utils import ellipsoid_gsd #note this is the development version of cmeutils

GSD_PATH = 'trajectory.gsd'
lpar = lperp = 0.5
ellipsoid_gsd(
    gsd_file=GSD_PATH,
    new_file="visual-ellipsoid-dpd.gsd",
    ellipsoid_types="R",
    lpar=lpar,
    lperp=lperp,
)